<a href="https://colab.research.google.com/github/Ayush-Devs/Util/blob/main/NNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, precision_score, accuracy_score

In [112]:

df = pd.read_csv("unique_legal_queries.csv")

X = df["query"]
y = df["label_encoded"]
y_one_hot = to_categorical(y, num_classes=9)

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)
tfidf = TfidfVectorizer(max_features=99)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [113]:
X = df["query"]
y = df["label_encoded"]

y_one_hot = to_categorical(y, num_classes=9)

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

In [114]:
tfidf = TfidfVectorizer(max_features=500)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [115]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')  # 9 categories
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [116]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_tfidf, y_train,
                    epochs=20,
                    batch_size=16,
                    validation_split=0.1)

Epoch 1/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1762 - loss: 2.1782 - val_accuracy: 0.2500 - val_loss: 2.1140
Epoch 2/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2917 - loss: 2.0636 - val_accuracy: 0.2222 - val_loss: 1.9511
Epoch 3/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3069 - loss: 1.8634 - val_accuracy: 0.3056 - val_loss: 1.6839
Epoch 4/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4279 - loss: 1.6079 - val_accuracy: 0.6806 - val_loss: 1.3312
Epoch 5/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6665 - loss: 1.2437 - val_accuracy: 0.8750 - val_loss: 0.9917
Epoch 6/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7780 - loss: 0.8707 - val_accuracy: 0.8750 - val_loss: 0.7471
Epoch 7/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8194 - loss: 0.7065 - val_accuracy: 0.8611 - val_loss: 0.5804
Epoch 8/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8960 - loss: 0.4945 - val_accuracy: 0.8889 - val_loss:

In [117]:
loss, accuracy = model.evaluate(X_test_tfidf, y_test)
print(f"Test Accuracy: {accuracy:.2f}")
y_pred = model.predict(X_test_tfidf)
y_pred_thresholded = (y_pred > 0.5).astype(int)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9216 - loss: 0.1943 
Test Accuracy: 0.93
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [118]:
f1 = f1_score(y_test, y_pred_thresholded, average='micro')
print(f"F1 Score: {f1}")


precision = precision_score(y_test, y_pred_thresholded, average='micro')
print(f"Precision Score: {precision}")

F1 Score: 0.9329608938547486
Precision Score: 0.9382022471910112


In [88]:
queries = ["a 17 yr old boy killed 2 people with a speeding car"]
queries_tfidf = tfidf.transform(queries).toarray()
predictions = model.predict(queries_tfidf)


predicted_categories = predictions.argmax(axis=1)
print(predicted_categories)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[0]


In [93]:
model.save("NNM.keras")

In [94]:
import pickle
with open('NNM.pkl', 'wb') as f:
    pickle.dump(model, f)